In [30]:
import keras.backend as K
import numpy as np
import json
import shap
import tensorflow as tf
import warnings
import os
from PIL import Image

In [31]:
import random

In [37]:
dataset_name = "combined_dataset"
batch_size = 64

In [38]:
# Load the dataset

image_size = (255, 255)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_name,
    validation_split=0.2,
    subset="training",
    label_mode = 'int',
    labels = 'inferred',
    seed=1336,
    image_size=image_size,
    interpolation = 'bilinear',
    batch_size=batch_size,
    shuffle = True,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_name,
    validation_split=0.2,
    subset="validation",
    label_mode = 'int',
    labels = 'inferred',
    seed=1336,
    image_size=image_size,
    interpolation = 'bilinear',
    batch_size=batch_size,
    shuffle = True,
)


Found 16514 files belonging to 3 classes.
Using 13212 files for training.
Found 16514 files belonging to 3 classes.
Using 3302 files for validation.


In [39]:
# Get a batch of images and labels from the validation dataset
for images, _ in val_ds.take(1):
    background = images[:100]  # Use a subset of images as background for SHAP
    images = images[:10]  # Choose a few images for explanation

In [40]:
#Cargo el modelo
# Specify the path to your saved model file (.h5 or .hdf5 format)
model_path = 'old_models/jorgenet_private_dataset/modelo_entrenado.h5'
# Load the model
model = tf.keras.models.load_model(model_path)


In [41]:
def pipeline(image_path): 
 
 #preprrocess 
 img = tf.keras.preprocessing.image.load_img(image_path, target_size=(255,255))
 x = tf.keras.preprocessing.image.img_to_array(img)    
 x = np.expand_dims(x, axis=0)

 #Predict
 prediction = model.predict(x)
 predicted_class = np.argmax(prediction)
 return(predicted_class)




In [34]:
# dataset_folder = "private_dataset"

# categories = os.listdir(dataset_folder)
# X = []
# y = []

# for i, category in enumerate(categories):
#     category_folder = os.path.join(dataset_folder, category)
#     image_files = os.listdir(category_folder)
    
#     for image_file in image_files:
#         image_path = os.path.join(category_folder, image_file)
#         image = Image.open(image_path).resize((224, 224))  # Resize the image to match the input size of your model
#         image = np.array(image)
#         X.append(image)
#         y.append(i)


# X = np.array(X)
# y = np.array(y)

In [48]:

# Create a DeepExplainer with the background dataset
e = shap.DeepExplainer(model, background)

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


In [49]:
shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough

In [50]:
shap_values = e.shap_values(images)

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


ResourceExhaustedError:  OOM when allocating tensor with shape[128,32,254,254] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_5/conv2d_15/Conv2D (defined at C:\Users\diosdadj\AppData\Roaming\Python\Python39\site-packages\shap\explainers\_deep\deep_tf.py:247) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_grad_graph_8664]

Function call stack:
grad_graph
